In [215]:
data_path = "../Data/Day_6_input.txt"
with open(data_path) as file:
    lines = [line.rstrip() for line in file]

## Part 1

In part 1 I need to trace the path of the guard and record how many steps they will take before leaving the grid.

In [216]:
# Read the grid as a list of lists for easy iteration
grid = []

for line in lines:
    grid.append([letter for letter in line])

In [217]:
# Create a grid of blank spaces to record the path taken
nrow = len(grid) - 1
ncol = len(grid[0]) - 1

row = []

for i in range(ncol + 1):
    row.append('.')

path_grid = []
for i in range(nrow + 1):
    path_grid.append(row.copy())

In [218]:
# Function to check if a coordinate is valid given the grid size, to prevent index out of bounds erros
def check_valid(row, col, nrow, ncol):
    valid = True
    if row < 0:
        valid = False
    if row > nrow:
        valid = False
    if col < 0:
        valid = False
    if col > ncol:
        valid = False
    return valid

In [219]:
# Function to get the characted from a particular space in a grid
def check_space(grid, row, col):
    char = grid[row][col]
    return char

In [220]:
# Function to turn 90 degrees to the right
def rotate(in_char):
    if in_char == "^":
        out_char = ">"
    elif in_char == ">":
        out_char = "v"
    elif in_char == "v":
        out_char = "<"
    elif in_char == "<":
        out_char = "^"
    else:
        raise ValueError('Input direction invalid.')
        
    return out_char

In [221]:
# Function to get the next row and column positions based on the direction we are facing
def get_next_pos(row, col, in_char):
    if in_char == "^":
        next_row = row - 1
        next_col = col
    elif in_char == ">":
        next_row = row
        next_col = col + 1
    elif in_char == "v":
        next_row = row + 1
        next_col = col
    elif in_char == "<":
        next_row = row
        next_col = col - 1
    else:
        raise ValueError('Input direction invalid.')
        
    return next_row, next_col

In [222]:
# Possible directions to look for to determine start position in grid
dirs = ["<", "^", ">", "v"]

In [223]:
# Find the starting position
for row_num in range(len(grid)):
    row = grid[row_num]
    for col_num in range(len(row)):
        if grid[row_num][col_num] in dirs:
            start_row = row_num
            start_col = col_num
            break

In [224]:
start_direction = grid[start_row][start_col]

In [225]:
start_direction

'^'

In [226]:
direction = start_direction
row = start_row
col = start_col
on_grid = True
path_coords = [[row,col]]
path_grid[row][col] = "X" # mark the start position in the blank grid
moves = 0
while on_grid: # while we are still in the map area
    # Check what the next step will be based on our current step and direction we are facing
    next_row, next_col = get_next_pos(row, col, direction)
    # Check if we are still on the map
    on_grid = check_valid(next_row, next_col, nrow, ncol)
    if on_grid:
        # If next position is valid, check if it is an obstacle
        next_char = check_space(grid, next_row, next_col)
        if next_char == "#":
            # If we hit an obstacle, rotate direction and keep same row and column
            direction = rotate(direction)
        else:
            # Otherwise move to the next space
            row = next_row
            col = next_col
            path_grid[row][col] = 'X' # mark the step in the path grid
            moves += 1
            if not [row,col] in path_coords:
                path_coords.append([row,col])

In [227]:
len(path_coords)

5329

In [228]:
path_grid

[['.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.'],
 ['.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.',
  '.'

In [229]:
# Record how many moves were needed for the actual path
actual_moves = moves

## Part 2

In part 2 we decide we actually want to trap the guard in a loop by placing a single new obstacle in the grid. The goal is to count how many locations on the grid we could do this successfully.

There is bound to be a smarter way to do this, but I will tackle it by iterating every grid space and creating a new grid each time with the new obstacle placed. Then I will wrap the previous code in a function that returns True if the space is a possible place to trap the guard in a loop. Since the function used a while loop, I will add a step count to break the loop if the steps surpass 150% of the actual path length (I am using this increase as a proxy way of concluding the guard is stuck in a loop).

In [242]:
def follow_path(grid, start_row, start_col, start_direction, actual_moves):
    direction = start_direction
    row = start_row
    col = start_col
    on_grid = True
    moves = 0
    loop_obstacle = False
    while on_grid:
        # Get the next position and check it will be valid on the grid
        next_row, next_col = get_next_pos(row, col, direction)
        on_grid = check_valid(next_row, next_col, nrow, ncol)
        if on_grid:
            # If next position is valid, check if it is an obstacle
            next_char = check_space(grid, next_row, next_col)
            if next_char == "#":
                # If we hit an obstacle, rotate and keep same row and column
                direction = rotate(direction)
            else:
                # Otherwise move to the next space
                moves += 1
                row = next_row
                col = next_col
        # If we have taken more than the step limit we applied, conclude we are stuck in a loop and break
        if moves > actual_moves:
            loop_obstacle = True
            break
    return loop_obstacle

Iterate through the whole grid, create a new grid every time and change a single space to an obstacle. Then check if following the path results in a 50% or more increase in moves for the guard (I will make the assumption they are stuck in a loop then).

In [243]:
possible_obstacles = 0
step_increase_mult = 1.5 # stop at 50% increase in steps
for row in range(nrow + 1):
    for col in range(ncol + 1):
        # New grid need to be a list of copies of the rows from the real grid
        new_grid = [row.copy() for row in grid]
        # Change a space to an obstacle (check if it already is obstacle and skip if so)
        if new_grid[row][col] == "#":
            continue
        new_grid[row][col] = "#"
        # Check if this new obstacle location is a possible loop location
        possible = follow_path(new_grid, start_row, start_col, start_direction, actual_moves*step_increase_mult) 
        # Add to count if it is a possible location
        if possible:
            possible_obstacles += 1
        

In [244]:
possible_obstacles

2162